In [1]:
import pandas as pd
from collections import Counter

In [2]:
df_sens = pd.read_csv("sensitive_topics_vs_source.csv")

In [3]:
Counter(df_sens['source'])

Counter({'toloka': 9291, 'trtest': 378, 'self': 1978, 'kw': 21656})

In [4]:
topics = ['offline_crime', 'online_crime', 'drugs', 'gambling', 'pornography',
       'prostitution', 'slavery', 'suicide', 'terrorism', 'weapons',
       'body_shaming', 'health_shaming', 'politics', 'racism', 'religion',
       'sexual_minorities', 'sexism', 'social_injustice']

In [5]:
df_sens.head()

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,source
0,РРРРРРЯЯЯЯЯЯЯ РУССКИЕ ВАНЬКИ НИКОМУ НЕ НУЖНЫ!!...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,toloka
1,"По моему, быдло тот кто осуждает чужие предпоч...",0.0,0.0,0.0,0.0,1.0,0.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,toloka
2,Бабы это вообще безхуйные сиськастые ссущие си...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,trtest
3,изнасиловалиа говорила что девственницаЯ так д...,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,trtest
4,"и что ? не сосут только уебищные тни, видосов ...",0.0,0.0,0.0,0.0,1.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,toloka


In [6]:
def get_self_check_df(target_df):
    collected_df_list = []
    for t in topics:
        df_c = target_df[target_df[t] > 0.8][:10]
        topics_curr = [t] * len(df_c)
        df_c['topic'] = topics_curr
        df_c = df_c[['text','topic']]
        collected_df_list.append(df_c)
    df_check = pd.concat(collected_df_list)      
    return df_check
    

In [7]:
df_sens_kw = df_sens[df_sens['source'] == 'kw']

In [8]:
df_sens_kw_for_check = get_self_check_df(df_sens_kw)

In [9]:
len(df_sens_kw_for_check), Counter(df_sens_kw_for_check['topic'])

(180,
 Counter({'offline_crime': 10,
          'online_crime': 10,
          'drugs': 10,
          'gambling': 10,
          'pornography': 10,
          'prostitution': 10,
          'slavery': 10,
          'suicide': 10,
          'terrorism': 10,
          'weapons': 10,
          'body_shaming': 10,
          'health_shaming': 10,
          'politics': 10,
          'racism': 10,
          'religion': 10,
          'sexual_minorities': 10,
          'sexism': 10,
          'social_injustice': 10}))

In [14]:
df_sens_kw_for_check.to_csv("check_keywords_lables.csv",index = None)

In [10]:
df_self = df_sens[df_sens['source'] == 'self']

In [11]:
df_self_for_check = get_self_check_df(df_self)

In [12]:
len(df_self_for_check), Counter(df_self_for_check['topic'])

(101,
 Counter({'offline_crime': 10,
          'online_crime': 10,
          'drugs': 10,
          'gambling': 1,
          'pornography': 10,
          'prostitution': 1,
          'slavery': 2,
          'suicide': 10,
          'terrorism': 2,
          'weapons': 4,
          'body_shaming': 1,
          'health_shaming': 10,
          'politics': 9,
          'sexual_minorities': 1,
          'sexism': 10,
          'social_injustice': 10}))

In [13]:
df_self_for_check.to_csv("check_self_lables.csv",index = None)